In [1]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import pickle

In [2]:
# activation function
def sigmoid(x):
    return 1 / (1 + jnp.exp(-x))

def relu(x):
    return jnp.maximum(0, x)

def softmax(x):
    c = jnp.max(x)
    exp_x = jnp.exp(a-c)
    sum_exp_x = jnp.sum(exp_x)
    y = exp_x / sum_exp_x

    return y

def softmax_batch(x):
    x = x - jnp.max(x, axis=-1, keepdims=True)   # オーバーフロー対策
    return jnp.exp(x) / jnp.sum(jnp.exp(x), axis=-1, keepdims=True)

# test
x = jnp.array([-1.0, 1.0, 2.0])
print(jnp.allclose(sigmoid(x), jnp.array([ 0.26894142, 0.73105858, 0.88079708])))

a = jnp.array([0.3, 2.9, 4.0])
print(jnp.allclose(jnp.sum(softmax(x)), 1))

True
True


In [3]:
from dataset.mnist import load_mnist
# 第一次调用会花费几分钟......
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
# 输出各个数据的形状 
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000,) 
print(x_test.shape) # (10000, 784) 
print(t_test.shape) # (10000,)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [4]:
import numpy as np
from dataset.mnist import load_mnist 
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img)) 
    display(pil_img)

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]
print(label) # 5
print(img.shape) # (784,)
img = img.reshape(28, 28) # 把图像的形状变成原来的尺寸 
print(img.shape) # (28, 28)
img_show(img)

5
(784,)
(28, 28)


In [5]:
def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, flatten=True, one_hot_label=False) 
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3'] 
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = jnp.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = jnp.dot(z1, W2) + b2 
    z2 = sigmoid(a2)
    a3 = jnp.dot(z2, W3) + b3
    y = softmax_batch(a3)
    return y

In [6]:

x, t = get_data() 
network = init_network()
batch_size = 100 # 批数量
accuracy_cnt = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352
